In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
# from src.load_transform_data import get_new_ethereum_ohlc, get_new_ethereum_ohlc_2
from src.hopsworks_connections import pull_data, upload_data, pull_model
import numpy as np
import requests
from datetime import datetime, timedelta
import time

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Since it is impossible to get 5-minute, or even hourly historical crypto data without paying alot for an API subscription, I made this pipeline to continously collect the last day's 5-minute data, and upload it to hopsworks feature store. It can be used eventually to get to train a model once there is enough data.

In [1]:
def fetch_recent_data(crypto):
    # 20:00, or 8pm in EST is 12:00 in UTC, which is the cycle by which crypto OHLC are defined
    # end_date = datetime.now().replace(hour=20, minute=0, second=0, microsecond=0)
    # ^^^ ONLY WHEN RUNING LOCALLY, on github actions this will retrieve different data, so just use datetime.now() and schedule the run at 8pm EST = 0:00 UTC
    end_date = datetime.now()
    start_date = end_date - timedelta(days=1)

    url = f"https://api.coingecko.com/api/v3/coins/{crypto}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': int(start_date.timestamp()),
        'to': int(end_date.timestamp())
    }
    
    response = safe_request(url, params)
    
    if response and response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['date', 'price'])
        df['date'] = pd.to_datetime(df['date'], unit='ms')
        df['market_cap'] = pd.DataFrame(data['market_caps'])[1].values
        df['volume_24h'] = pd.DataFrame(data['total_volumes'])[1].values
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}" if response else "Failed to fetch data; no response.")
    
    # Include only the earliest day
    df['date'] = pd.to_datetime(df['date'])
    # Step 2: Extract the date part from the 'datetime' column
    df['no_hour_date'] = df['date'].dt.date
    earliest_date = df['no_hour_date'].min()
    filtered_df = df[df['no_hour_date'] == earliest_date]
    filtered_df = filtered_df.drop(columns='no_hour_date')
    
    return filtered_df

def safe_request(url, params, retries=20, backoff_factor=0.5):
    for i in range(retries):
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response
            elif response.status_code == 429:
                # We're being rate-limited; back off and retry
                sleep_time = backoff_factor * (2 ** i)
                print(f"Rate limit hit. Waiting {sleep_time:.2f} seconds before retrying...")
                time.sleep(sleep_time)
            else:
                # Other errors, break the retry loop and return None
                print(f"Request failed with status code {response.status_code}.")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request exception: {e}. Retrying...")
            time.sleep(backoff_factor * (2 ** i))

    return None

In [14]:
def get_top_coins_by_market_cap(): # 5256000
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 25,
        'page': 1,
        'sparkline': False
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        # Extract coin names and market caps
        coins_info = [(coin['id'], coin['market_cap']) for coin in data]
        return coins_info
    else:
        return f"Failed to retrieve data: {response.status_code}"
    
# Example usage
top_coins = get_top_coins_by_market_cap()
for coin_name, market_cap in top_coins:
    print(coin_name, market_cap)

bitcoin 1265699266592
ethereum 383647821804
tether 110366566872
binancecoin 93931760217
solana 64256810527
usd-coin 33357510296
staked-ether 29417207955
ripple 29063691769
dogecoin 21725876680
the-open-network 18666120812
cardano 16701195506
shiba-inu 15179271796
avalanche-2 13444724672
tron 10297940663
wrapped-bitcoin 9986741003
bitcoin-cash 9373655589
polkadot 9291260914
chainlink 8613059752
near 7904626219
matic-network 6712641842
internet-computer 6330638524
litecoin 6315400395
uniswap 5915807358
leo-token 5319068654
dai 5192740779


In [12]:
eth_raw = fetch_recent_data("usd coin")
eth_raw

Request failed with status code 404.
Failed to fetch data; no response.


UnboundLocalError: local variable 'df' referenced before assignment